# CS4487 Machine Learning | City University of Hong Kong
## Group Assignment - until 2019.12.13
- Tim Löhr       , EID: 40126684 
- Timo Bohnstedt , EID:
- Karina         , EID:

### Try to get the best possible accuracy for the Cifar 10 task

In [23]:
# Keras bib's
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10

import numpy as np
import matplotlib.pyplot as plt
from scipy.misc.pilutil import imread

In [21]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [22]:
x_train.shape

(50000, 32, 32, 3)